# IEOR 174 Assignment 1: Deliverable 3
Jacob Quisumbing

Joy Zhang

In [160]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

In [140]:
start_time = 0  # 11 pm in min 
end_time = 120  # 1 pm in min

arrival_rate_start = 0.5  # arrival rate at 11
arrival_rate_end = 1.0    # arrival rate at 1
service_time_mean = 35 / 60  # average service time in minutes

num_replications = 100  # number of simulation replications
confidence_level = 0.95  # for the confidence interval

From the given arrival rates, we can write the arrival rate as a function of time: 
$$ \lambda(t) = 0.5 + \frac{0.5}{120} t$$
Simplifying, we get: 
$$ \lambda(t) = 0.5 + \frac{1}{240} t $$

In [141]:
def simulate_arrivals(start_time, end_time, arrival_rate_start, arrival_rate_end):
    arrivals = []
    current_time = start_time
    while current_time < end_time:
        t = arrival_rate_start + (arrival_rate_end - arrival_rate_start) * (current_time - start_time) / (end_time - start_time)
       
        # Simulate the next arrival
        inter_arrival_time = np.random.exponential(1 / t)
        current_time += inter_arrival_time
        if current_time < end_time:
            arrivals.append(current_time)
    return arrivals

In [142]:
# simulating the service process
def simulate_service(service_time_mean):
    return np.random.exponential(service_time_mean)

In [149]:
def get_arrival_rate(t):
    return arrival_rate_start + (arrival_rate_end - arrival_rate_start) * t / (end_time - start_time)
    
def simulate_whole_day():
    current_time = start_time
    customers_in_system = 0
    next_arrival_time = np.random.exponential(1 / get_arrival_rate(0))  # first arrival
    next_departure_time = np.inf  # no customers initially, so no departure
    
    num_customers_at_1pm = 0

    while current_time < end_time:
        if next_arrival_time < next_departure_time:  # Arrival event
            current_time = next_arrival_time
            customers_in_system += 1
            next_arrival_time += np.random.exponential(1 / get_arrival_rate(current_time))
            if customers_in_system == 1:  # If server is idle, start service
                next_departure_time = current_time + np.random.exponential(service_time_mean)
        else:  # Departure event
            current_time = next_departure_time
            customers_in_system -= 1
            if customers_in_system > 0:
                next_departure_time = current_time + np.random.exponential(service_time_mean)
            else:
                next_departure_time = np.inf

        # Record the number of customers at 1PM (time = 120)
        if current_time >= end_time and customers_in_system >= 0:
            num_customers_at_1pm = customers_in_system

    return num_customers_at_1pm

In [151]:
def run_simulation(num_replications, start_time, end_time, arrival_rate_start, arrival_rate_end, service_time_mean):
    waiting_times_2 = []
    waiting_times_3 = []

    for _ in range(num_replications):
        # Reset the state at the start of each replication
        arrivals = simulate_arrivals(start_time, end_time, arrival_rate_start, arrival_rate_end)
        server_free_time = 0
        waiting_times = []

        for arrival in arrivals:
            service_time = simulate_service(service_time_mean)
            if arrival >= server_free_time:
                waiting_time = 0
                server_free_time = arrival + service_time
            else:
                waiting_time = server_free_time - arrival
                server_free_time += service_time
            waiting_times.append(waiting_time)

        # Calculate average waiting times for specific time intervals
        arrivals_2 = [waiting_times[i] for i, t in enumerate(arrivals) if 105 <= t < 120]  # 12:45 PM to 1 PM
        if arrivals_2:
            waiting_times_2.append(np.mean(arrivals_2))

        arrivals_3 = [waiting_times[i] for i, t in enumerate(arrivals) if 45 <= t < 60]  # 11:45 AM to 12 PM
        if arrivals_3:
            waiting_times_3.append(np.mean(arrivals_3))

    return waiting_times_2, waiting_times_3


In [152]:
#computing confidence intervals
def compute_confidence_interval(data, confidence_level):
    mean = np.mean(data)
    stderr = st.sem(data)  # standard error of the mean
    margin_of_error = stderr * st.t.ppf((1 + confidence_level) / 2, len(data) - 1)
    return mean, mean - margin_of_error, mean + margin_of_error


## Part A

In [162]:
all_1pm_cust_counts = []
for day in range(num_replications):
    count_at_1pm = simulate_whole_day()
    all_1pm_cust_counts.append(count_at_1pm)

part_a = np.mean(all_1pm_cust_counts)

part_a_ci = compute_confidence_interval(all_1pm_cust_counts, confidence_level)

print(f"Expectation of the number of Customers at 1pm: {part_a} customers") 
print(f"95% confidence interval: {part_a_ci}")

Expectation of the number of Customers at 1pm: 1.42 customers
95% confidence interval: (1.42, 1.1871291895984941, 1.6528708104015057)


## Part B and C

In [163]:
#running simulation
waiting_times_2, waiting_times_3 = run_simulation(
    num_replications, start_time, end_time, arrival_rate_start, arrival_rate_end, service_time_mean)


#mean and confidence intervals for each part
mean_waiting_2, ci_low_waiting_2, ci_high_waiting_2 = compute_confidence_interval(waiting_times_2, confidence_level)
mean_waiting_3, ci_low_waiting_3, ci_high_waiting_3 = compute_confidence_interval(waiting_times_3, confidence_level)

print(f"Avg waiting time for 12:45 to 1: Mean = {mean_waiting_2:.2f} minutes, 95% CI = [{ci_low_waiting_2:.2f}, {ci_high_waiting_2:.2f}]")
print(f"Avg waiting time for 11:45 to 12:00: Mean = {mean_waiting_3:.2f} minutes, 95% CI = [{ci_low_waiting_3:.2f}, {ci_high_waiting_3:.2f}]")


Avg waiting time for 12:45 to 1: Mean = 0.64 minutes, 95% CI = [0.49, 0.79]
Avg waiting time for 11:45 to 12:00: Mean = 0.36 minutes, 95% CI = [0.28, 0.44]
